<a href="https://colab.research.google.com/github/Aleksvw/eKConstruct/blob/main/Social_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive document will retrieve and process Social Media Data.

###Initial Packages. Run these Cells together.

In [ ]:
library(tidyverse)
#install.packages("googledrive")
#install.packages("httpuv")
#library("googledrive")
#library("httpuv")
#install.packages("R.utils")
#library("R.utils")
library("httr")
#my_check <- function() {return(TRUE)}
#reassignInPackage("is_interactive", pkgName = "httr", my_check)
#options(rlang_interactive=TRUE)
#drive_auth(use_oob = TRUE, cache = TRUE)

In [ ]:
#drive_download(as_id("1ZeRNJR1E3M1KYj41Us1aZYQCecIbMkR4"))
#auth <- readLines("tok1.txt") %>% suppressWarnings()
#main_id <- auth[1] %>% as.character()
#main_token <- auth[3] %>% as.character()
# Translating data to and from JSON format
library(tidyverse)
library(jsonlite)
# My FB Account ID
main_id <- "10219064988330003"

# Facebook Graph API Token, set-up on developers.facebook.com
# This one is set to never expire, which requires setup also.
main_token <-
  "EAAFFpgGNYd0BACgdvagncTt3MV7ZAqi49Bd5ff5gz0ys2X9XOVkr1jYxIbAsOVJAZBjleA7ZB5iT
4WP3VLBW5VjOFGjkHKbWJAyX8VuLM0IaOJWfwJN1H4NlYa2srlriEJ6He6yR5LogjW3SxNYi2X08UO33
goGIcPHZCWl1XGPZCA9j5GtTjIMB2c0fXQLMZD"

main_token <-
  "EAAFFpgGNYd0BACgdvagncTt3MV7ZAqi49Bd5ff5gz0ys2X9XOVkr1jYxIbAsOVJAZBjleA7ZB5iT
4WP3VLBW5VjOFGjkHKbWJAyX8VuLM0IaOJWfwJN1H4NlYa2srlriEJ6He6yR5LogjW3SxNYi2X08UO33
goGIcPHZCWl1XGPZCA9j5GtTjIMB2c0fXQLMZD"


#main_id <- "580970248242039"
#main_token <- "EAAFFpgGNYd0BO7ZCXaRaDKBQrcqGDzSzAagupKivPozhBYoD2ZBNwWacZCuk8Q5oOZCv5X9HKhW0ZBXr5B6A1GvV9zg0YZA0wLZB87Hnc1TBYCbEZAdcaxMj1wiCrdvnkoxKyKZBbatDK5C4HZC8QDUDKZBPh9SXYY8lAixtlHGRAAlxjfiHmqthvDHte2TZBHLyshVu5U5dvtNh"

### Set Parameters: Select Dates and Accounts to Process
Here you need to specify the dates and the clients to process. This is setup to handle two months at a time, and would need slight modification to handle more segments of time.

**Note** that you shouldn't run this section all at once. You need to view the List of available accounts first, and then enter the relevant IDs in the following cell to query the correct accounts.

In [ ]:
# These set the starting and ending times and folder name
starting <- "Mar 1 2025"
ending <- "Apr 30 2025"
monthlabel <- "Apr_Reps_1"

In [ ]:
get_fb_ids <- function(acc_id, access_token) {
  fb_ids <-
    GET(
      "https://graph.facebook.com/",
      path = paste(acc_id, "/", "accounts", sep = ""),
      query = list("access_token" = access_token)
    ) %>%
    content() %>%
    toJSON() %>%
    fromJSON()

  fb_ids <- fb_ids$data %>%
    select(name, id, access_token)

  return(fb_ids)
}

get_IG_ids <- function(acc_id, access_token) {
  input_FB_ids <- get_fb_ids(acc_id, access_token)

  ig_ids <- lapply(1:nrow(input_FB_ids), function(i) {
    q_time <- proc.time()
    d <- GET(
      "https://graph.facebook.com/",
      path = (input_FB_ids[i, "id"]),
      query = list("access_token" = access_token,
                   "fields" = "instagram_business_account")
    ) %>%
      content() %>%
      toJSON() %>%
      fromJSON()

    d <- d[["instagram_business_account"]] %>% unlist()

    if (is_null(d)) {
      d <- "No Object"
    }
    q_time <- proc.time() - q_time
    r <-
      c(ig_bs_id = d,
        b_name = input_FB_ids[i, "name"],
        time = q_time["elapsed"])
    names(r) <- c("ig_bs_id", "b_name", "time")
    return(r)
  }) %>% bind_rows()

  no_IG <- ig_ids %>% filter(ig_bs_id == "No Object")
  active_IG_ids <- ig_ids %>% filter(ig_bs_id != "No Object")
  print(no_IG)
  return(active_IG_ids)
}

print("Note: the ids that say \"no object\" do not appear to have IG business access")
iL_init <- get_IG_ids(main_id, main_token)
fL_init <- get_fb_ids(main_id, main_token)

iL_init
fL_init

Enter the ID numbers, separated by commas, from the Accounts to query list below:

In [ ]:
#IG IDs to use
iL <- iL_init[c(1),]

#FB IDs to use
fL <- fL_init[c(1),]


# Load Functions
Run these All Together

## FB Functions

In [ ]:
# FB page insights --------------------------------------------------------
install.packages("beepr")
get_FB_profile_insights <- function(input_FB_ids, intervals,
                                    reportMonth = "TestReport") {
  #Get Media and insights####

  # Get Page insights####
  FB_insights <- lapply(1:nrow(input_FB_ids), function(j) {
    #From original FB
    get_FB_media <- function(input_FB_ids, intervals_FB#, reportMonth = "P"
                             ) {

      #Main Actions
      intervals_FB <- get_pages_fb(start_date = starting,end_date = ending)
      FB_media <- lapply(1:nrow(input_FB_ids), function(j) {
        # print(paste("j:", j))
        #Media Ids for current account j
        FB_media_interval <- lapply(1:nrow(intervals_FB), function(i){
          # print(paste("i:", i))
          media_ids <- GET(
            "https://graph.facebook.com/",
            path = paste0(input_FB_ids$id[j], "/", "posts/"),
            query = list(
              # "metric" = paste(collapse = ", ", params),
              "fields" = "attachments,insights.metric(post_impressions_unique,post_impressions,post_engaged_users)",
              "access_token" = input_FB_ids$access_token[j],
              # period = "lifetime",
              since = intervals_FB$unix_min[i],
              until = intervals_FB$unix_max[i]
            )
          ) %>%
            content() %>%
            toJSON() %>%
            fromJSON()
          mids <- media_ids$data$id %>% unlist()
          if(!is.null(mids)){

            FB_media_page <- lapply(1:length(mids), function(k) {
              # print(paste("k:", k))
              params <-
                c("post_engaged_users","post_clicks", "post_impressions", "post_impressions_unique", "post_reactions_like_total"#, "video_views"
                )

              media_info <- GET(
                "https://graph.facebook.com/",
                path = paste0(mids[k], "/"),
                query = list(
                  "fields" = "id,full_picture,message,created_time",
                  "metric" = paste(collapse = ", ", params),
                  "access_token" = input_FB_ids$access_token[j],
                  period = "lifetime",
                  since = intervals_FB$unix_min[i],
                  until = intervals_FB$unix_max[i]
                )
              ) %>%
                content() %>%
                toJSON() %>%
                fromJSON() %>%
                bind_cols()

              if(!("message") %in% names(media_info)){
                media_info$message <- "No Message"
              }


              ins <- GET(
                "https://graph.facebook.com/",
                path = paste0(mids[k], "/", "insights/"),
                query = list(
                  "metric" = paste(collapse = ", ", params),
                  "access_token" = input_FB_ids$access_token[j],
                  period = "lifetime",
                  since = intervals_FB$unix_min[i],
                  until = intervals_FB$unix_max[i]
                )
              ) %>%
                content() %>%
                toJSON() %>%
                fromJSON() %>%
                bind_rows()

              ins <- ins[-c(nrow(ins)),]%>%
                mutate(values = unlist(values), title = as.character(title)) %>%
                select(c(title, values)) %>% t() %>% data.frame()
              names(ins) <- ins[1, ]
              ins <- ins[-1, ]

              out_val <- bind_cols(media_info, ins) %>%
                mutate(across(!(1:4), as.integer), created_time = as_datetime(created_time))
              # print(paste(j,k))
              return(out_val)
            }) %>% bind_rows()
          } else {

          }
        }) %>%  rbindlist(fill = T)
        # glimpse(FB_media_interval)
        ##Save Tables####
        FB_media_sum <- FB_media_interval %>%
          group_by(Month = lubridate::month(FB_media_interval$created_time, label = T)) %>%
          summarize(across(.cols = where(is.numeric), .fns = sum)) %>%
          mutate(`Engagement Rate` = (`Lifetime Engaged Users` / `Lifetime Post Total Reach`)) %>%
          t() %>%
          data.frame()

        if(ncol(FB_media_sum)>1){
          names(FB_media_sum) <- c(FB_media_sum[1,])
          FB_media_sum <- FB_media_sum[-1,] %>%
            mutate(across(where(is.character), as.numeric))
          corrected_metrics <- rownames(FB_media_sum)
          corrected_metrics <- gsub("_", " ", corrected_metrics)
          corrected_metrics <- gsub("Lifetime ", "", corrected_metrics) %>%
            stringr::str_to_title()
          FB_media_sum$Change <- FB_media_sum[,2] - FB_media_sum[,1]
          FB_media_sum[rownames(FB_media_sum) !="Engagement Rate",] <- FB_media_sum[rownames(FB_media_sum) !="Engagement Rate",] %>%
            mutate(across(everything(),scales::comma,accuracy = 1))
          FB_media_sum["Engagement Rate",] <- scales::percent(as.numeric(FB_media_sum["Engagement Rate",]), accuracy = 0.01)
          FB_media_sum <- cbind(Metric = corrected_metrics, FB_media_sum)

          # glimpse(FB_media_sum)
          # FB_media_sum$Change <- 0
          # FB_media_sum <- bind_cols(rownames(FB_media_sum),FB_media_sum)
          #
          # Change <- FB_media_sum[,3] - FB_media_sum[,2]




          clientName = input_FB_ids$name[[j]]

          #need to fix this to be arbitrary
          # reportMonth = "Jul"

          #temp test section
          mainDir =
            (getwd())
          subDir =
            paste(reportMonth,
                  "/",
                  clientName,
                  sep = "")
          #Folder Prep####
          #ensure folder exists, then
          prep_FB_media_folder <-
            function(clientName,
                     reportMonth = lubridate::month(Sys.time(), label = T),
                     mainDir,
                     subDir) {
              dir.create(file.path(mainDir, subDir, "FBOutput", "Media"),
                         recursive = TRUE)

              file_p_media <- file.path(
                mainDir,
                subDir,
                "FBOutput",
                "Media",
                paste(clientName, "FB", reportMonth, "media_info.csv", sep = "_")
              )

              file_p_media_ins <- file.path(
                mainDir,
                subDir,
                "FBOutput",
                "Media",
                paste(clientName, "FB", reportMonth, "media_summary.csv", sep = "_")
              )
              write_csv(FB_media_interval, file_p_media)
              write_csv(FB_media_sum, file_p_media_ins)
            }
          prep_FB_media_folder(
            clientName = clientName,
            reportMonth = reportMonth,
            mainDir = mainDir,
            subDir = subDir
          )

          dl_media <- lapply(1:nrow(FB_media_interval), function(p){
            # if(FB_media_interval$media_type[p]=="VIDEO"){
            #   ftype <- ".mp4"
            # } else {
            #   ftype <- ".jpg"
            # }
            ftype <- ".jpg"
            file_p <- file.path(
              mainDir,
              subDir,
              "FBOutput",
              "Media",
              paste(clientName, "FB", reportMonth,as.numeric(as.POSIXct(FB_media_interval$created_time[p])) ,date(FB_media_interval$created_time[p]) , ftype, sep = "_")
            )
            download(FB_media_interval$full_picture[p], file_p, mode="wb")

          })

        }

      })
      return(FB_media)


    }

    # fbtb <- fL[c(1),]
    #fbtb <- input_FB_ids
    #FB_Media <-
     # get_FB_media(fbtb, get_pages_fb(starting, ending)#, reportMonth
      #             )

    FB_insight_page <- lapply(1:nrow(intervals), function(i) {
      params <- c("page_post_engagements", "page_consumptions_unique", "page_impressions",
                  "page_impressions_paid",
                  # "page_impressions_organic",
                  "page_impressions_viral",
                  "page_impressions_nonviral","page_impressions_unique","page_video_views", "page_fan_adds",
                  "page_fan_removes")
      ins <- GET(
        "https://graph.facebook.com/",
        path = paste0(input_FB_ids$id[j], "/", "insights/"),
        query = list(
          "metric" = paste(collapse = ", ", params),
          "access_token" = input_FB_ids$access_token[j],
          period = "day",
          since = intervals$unix_min[i],
          until = intervals$unix_max[i]
        )
      ) %>%
        content() %>%
        toJSON() %>%
        fromJSON()

      for (n in 1:length(params)) {
        ins$data$values[[n]] <- data.frame(ins$data$values[[n]])

        names(ins$data$values[[n]])[1] <- gsub("Daily ", "",unlist(ins$data$title)[[n]])
      }

      out_val <- (ins$data$values %>%
                    reduce(inner_join, by = "end_time")) %>%
        select(end_time, everything()) %>%
        mutate(end_time=as_date(as.character(end_time))-1
        ) %>%
        mutate_if(is.list, as.numeric) %>%
        mutate(`Net Likes` = `New Likes` - Unlikes
        )
      names(out_val)[1] <- "Date"

      return(out_val)
    }) %>% bind_rows()

    #Folder Prep####
    prep_FB_folders <-
      function(clientName,
               reportMonth = lubridate::month(Sys.time(), label = T),
               mainDir,
               subDir) {
        dir.create(file.path(mainDir, subDir, "FBInput"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Daily"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Faceted"),
                   recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Media"), recursive = TRUE)

        file_p <- file.path(mainDir,
                            subDir,
                            "FBInput",
                            paste(clientName, "FB", reportMonth, ".csv", sep = "_"))
        # file.copy(in_file, file_p)
        write_csv(FB_insight_page, file_p)
        return(file_p)
      }
    clientName = input_FB_ids$name[[j]]
    # reportMonth = lubridate::month(Sys.time(), label = T) %>%
    #   droplevels()
    # reportMonth = "Jul"

    #temp test section
    mainDir =
      (getwd())
    subDir =
      paste(
        reportMonth,
        "/",
        input_FB_ids$name[[j]],
        sep = ""
      )

    prep_FB_folders(
      clientName = clientName,
      reportMonth = reportMonth,
      mainDir = mainDir,
      subDir = subDir
    )

    # FB Summary ####
    # print(paste("Month Curr:", FB_insight_page$Date[1]))
    FB_sum <- FB_insight_page %>%
      mutate(Metric =lubridate::month(Date, label = T)) %>%
      group_by(Metric) %>%
      summarise(across(where(is.numeric), sum)) %>%
      mutate(`Engagement Rate` = (`Post Engagements`/`Total Reach`)) %>%
      t() %>%
      data.frame()
    # glimpse(FB_sum)
    names(FB_sum) <- FB_sum[1, ]
    FB_sum <- FB_sum[-1, ]
    FB_sum <- FB_sum %>%
      mutate_at(1:2, as.numeric)
    Change <- (FB_sum[, 2] - FB_sum[, 1])
    FB_sum$Change <- Change
    FB_sum[rownames(FB_sum) !="Engagement Rate",] <- FB_sum[rownames(FB_sum) !="Engagement Rate",] %>%
      mutate(across(everything(),scales::comma,accuracy = 1))
    FB_sum["Engagement Rate",] <- scales::percent(as.numeric(FB_sum["Engagement Rate",]), accuracy = 0.01)
    FB_sum <- cbind(Metric = rownames(FB_sum), FB_sum)

    file_sum_p <- file.path(
      mainDir,
      subDir,
      "FBOutput",
      paste(clientName, "FB_Summary", reportMonth, ".csv", sep = "_")
    )
    write_csv(FB_sum, file_sum_p)

    #FB Charts####

    #Fix Dates and Variables####
    fixDates <- function(inp) {
      allDate <- as_date(min(inp$Date):max(inp$Date))
      missingDates <- as_date(setdiff(allDate, inp$Date))
      fillDates <- data.frame(Date = missingDates)
      x <- bind_rows(inp, fillDates) %>%
        arrange(Date)
      x[is.na(x)] <- 0
      return(x)
    }

    FBTbl <- FB_insight_page %>%
      fixDates() %>%
      mutate(
        #Week = week(FBTbl$Date),
        Month = lubridate::month(Date, label = T),
        Day = day(Date),
        Weekday = lubridate::wday(Date, label = T)
      )
    FBTime <- c("Date", "Month", "Week", "Day", "Weekday")
    FBTime <- set_names(FBTime)
    FBVars <- names(FBTbl)
    FBVars <- FBVars[!FBVars %in% FBTime]
    FBVars <- set_names(FBVars)
    FBTbl <- FBTbl %>% droplevels()
    meanOfVars <- lapply(seq(levels(FBTbl$Month)), function(i) {
      x <- FBTbl %>%
        filter(FBTbl$Month == levels(FBTbl$Month)[i]) %>%
        select(all_of(FBVars))
      monthSum <- summarise_each(x, list(mean = mean))
      monthSum$Month <- levels(FBTbl$Month)[i]
      return(monthSum)
    }) %>% bind_rows()

    #Daily Charts####
    FB_charts_daily <- lapply(seq(FBVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- FBVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(FBTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(FBTbl[current_variable]), max(FBTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "FBOutput",
          "/","Daily","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })
    #Faceted Charts####
    FB_charts_faceted <- lapply(seq(FBVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- FBVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(FBTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(FBTbl[current_variable]), max(FBTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable) +
        facet_wrap(~Weekday)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "FBOutput",
          "/","Faceted","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })

    return(FB_insight_page)
  })
  names(FB_insights) <- input_FB_ids$name

  return(FB_insights)
}


In [ ]:
# FB page insights --------------------------------------------------------
install.packages("beepr")
get_FB_profile_insights <- function(input_FB_ids, intervals,
                                    reportMonth = "TestReport") {
  #Get Media and insights####

  # Get Page insights####
  FB_insights <- lapply(1:nrow(input_FB_ids), function(j) {
    #From original FB

    # fbtb <- fL[c(1),]
    #fbtb <- input_FB_ids
    #FB_Media <-
     # get_FB_media(fbtb, get_pages_fb(starting, ending)#, reportMonth
      #             )

    FB_insight_page <- lapply(1:nrow(intervals), function(i) {
      params <- c("page_post_engagements",
                  # "page_consumptions_unique",
                  "page_impressions",
                  "page_impressions_paid",
                  # "page_impressions_organic_unique",
                  "page_impressions_viral",
                  "page_impressions_nonviral","page_impressions_unique","page_video_views", "page_fan_adds",
                  "page_fan_removes")
      ins <- GET(
        "https://graph.facebook.com/",
        path = paste0(input_FB_ids$id[j], "/", "insights/"),
        query = list(
          "metric" = paste(collapse = ", ", params),
          "access_token" = input_FB_ids$access_token[j],
          period = "day",
          since = intervals$unix_min[i],
          until = intervals$unix_max[i]
        )
      ) %>%
        content() %>%
        toJSON() %>%
        fromJSON()

      for (n in 1:length(params)) {
        ins$data$values[[n]] <- data.frame(ins$data$values[[n]])

        names(ins$data$values[[n]])[1] <- gsub("Daily ", "",unlist(ins$data$title)[[n]])
      }

      out_val <- (ins$data$values %>%
                    reduce(inner_join, by = "end_time")) %>%
        select(end_time, everything()) %>%
        mutate(end_time=as_date(as.character(end_time))-1
        ) %>%
        mutate_if(is.list, as.numeric) %>%
        mutate(`Net Likes` = `New Likes` - Unlikes
        )
      names(out_val)[1] <- "Date"

      return(out_val)
    }) %>% bind_rows()

    #Folder Prep####
    prep_FB_folders <-
      function(clientName,
               reportMonth = lubridate::month(Sys.time(), label = T),
               mainDir,
               subDir) {
        dir.create(file.path(mainDir, subDir, "FBInput"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Daily"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Faceted"),
                   recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Media"), recursive = TRUE)

        file_p <- file.path(mainDir,
                            subDir,
                            "FBInput",
                            paste(clientName, "FB", reportMonth, ".csv", sep = "_"))
        # file.copy(in_file, file_p)
        write_csv(FB_insight_page, file_p)
        return(file_p)
      }
    clientName = input_FB_ids$name[[j]]
    # reportMonth = lubridate::month(Sys.time(), label = T) %>%
    #   droplevels()
    # reportMonth = "Jul"

    #temp test section
    mainDir =
      (getwd())
    subDir =
      paste(
        reportMonth,
        "/",
        input_FB_ids$name[[j]],
        sep = ""
      )

    prep_FB_folders(
      clientName = clientName,
      reportMonth = reportMonth,
      mainDir = mainDir,
      subDir = subDir
    )

    # FB Summary ####
    # print(paste("Month Curr:", FB_insight_page$Date[1]))
    FB_sum <- FB_insight_page %>%
      mutate(Metric =lubridate::month(Date, label = T)) %>%
      group_by(Metric) %>%
      summarise(across(where(is.numeric), sum)) %>%
      mutate(`Engagement Rate` = (`Post Engagements`/`Total Reach`)) %>%
      t() %>%
      data.frame()
    # glimpse(FB_sum)
    names(FB_sum) <- FB_sum[1, ]
    FB_sum <- FB_sum[-1, ]
    FB_sum <- FB_sum %>%
      mutate_at(1:2, as.numeric)
    Change <- (FB_sum[, 2] - FB_sum[, 1])
    FB_sum$Change <- Change
    FB_sum[rownames(FB_sum) !="Engagement Rate",] <- FB_sum[rownames(FB_sum) !="Engagement Rate",] %>%
      mutate(across(everything(),scales::comma,accuracy = 1))
    FB_sum["Engagement Rate",] <- scales::percent(as.numeric(FB_sum["Engagement Rate",]), accuracy = 0.01)
    FB_sum <- cbind(Metric = rownames(FB_sum), FB_sum)

    file_sum_p <- file.path(
      mainDir,
      subDir,
      "FBOutput",
      paste(clientName, "FB_Summary", reportMonth, ".csv", sep = "_")
    )
    write_csv(FB_sum, file_sum_p)

    #FB Charts####

    #Fix Dates and Variables####
    fixDates <- function(inp) {
      allDate <- as_date(min(inp$Date):max(inp$Date))
      missingDates <- as_date(setdiff(allDate, inp$Date))
      fillDates <- data.frame(Date = missingDates)
      x <- bind_rows(inp, fillDates) %>%
        arrange(Date)
      x[is.na(x)] <- 0
      return(x)
    }

    FBTbl <- FB_insight_page %>%
      fixDates() %>%
      mutate(
        #Week = week(FBTbl$Date),
        Month = lubridate::month(Date, label = T),
        Day = day(Date),
        Weekday = lubridate::wday(Date, label = T)
      )
    FBTime <- c("Date", "Month", "Week", "Day", "Weekday")
    FBTime <- set_names(FBTime)
    FBVars <- names(FBTbl)
    FBVars <- FBVars[!FBVars %in% FBTime]
    FBVars <- set_names(FBVars)
    FBTbl <- FBTbl %>% droplevels()
    meanOfVars <- lapply(seq(levels(FBTbl$Month)), function(i) {
      x <- FBTbl %>%
        filter(FBTbl$Month == levels(FBTbl$Month)[i]) %>%
        select(all_of(FBVars))
      monthSum <- summarise_each(x, list(mean = mean))
      monthSum$Month <- levels(FBTbl$Month)[i]
      return(monthSum)
    }) %>% bind_rows()

    #Daily Charts####
    FB_charts_daily <- lapply(seq(FBVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- FBVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(FBTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(FBTbl[current_variable]), max(FBTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "FBOutput",
          "/","Daily","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })
    #Faceted Charts####
    FB_charts_faceted <- lapply(seq(FBVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- FBVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(FBTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(FBTbl[current_variable]), max(FBTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable) +
        facet_wrap(~Weekday)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "FBOutput",
          "/","Faceted","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })

    return(FB_insight_page)
  })
  names(FB_insights) <- input_FB_ids$name

  return(FB_insights)
}


In [ ]:
# FB page insights --------------------------------------------------------
install.packages("beepr")
get_FB_profile_insights <- function(input_FB_ids, intervals,
                                    reportMonth = "TestReport") {
  #Get Media and insights####

  # Get Page insights####
  FB_insights <- lapply(1:nrow(input_FB_ids), function(j) {
    #From original FB
    get_FB_media <- function(input_FB_ids, intervals_FB#, reportMonth = "P"
                             ) {

      #Main Actions
      intervals_FB <- get_pages_fb(start_date = starting,end_date = ending)
      FB_media <- lapply(1:nrow(input_FB_ids), function(j) {
        # print(paste("j:", j))
        #Media Ids for current account j
        FB_media_interval <- lapply(1:nrow(intervals_FB), function(i){
          # print(paste("i:", i))
          media_ids <- GET(
            "https://graph.facebook.com/",
            path = paste0(input_FB_ids$id[j], "/", "posts/"),
            query = list(
              # "metric" = paste(collapse = ", ", params),
              "fields" = "attachments,insights.metric(post_impressions_unique,post_impressions,post_engaged_users)",
              "access_token" = input_FB_ids$access_token[j],
              # period = "lifetime",
              since = intervals_FB$unix_min[i],
              until = intervals_FB$unix_max[i]
            )
          ) %>%
            content() %>%
            toJSON() %>%
            fromJSON()
          mids <- media_ids$data$id %>% unlist()
          if(!is.null(mids)){

            FB_media_page <- lapply(1:length(mids), function(k) {
              # print(paste("k:", k))
              params <-
                c("post_engaged_users","post_clicks", "post_impressions", "post_impressions_unique", "post_reactions_like_total"#, "video_views"
                )

              media_info <- GET(
                "https://graph.facebook.com/",
                path = paste0(mids[k], "/"),
                query = list(
                  "fields" = "id,full_picture,message,created_time",
                  "metric" = paste(collapse = ", ", params),
                  "access_token" = input_FB_ids$access_token[j],
                  period = "lifetime",
                  since = intervals_FB$unix_min[i],
                  until = intervals_FB$unix_max[i]
                )
              ) %>%
                content() %>%
                toJSON() %>%
                fromJSON() %>%
                bind_cols()

              if(!("message") %in% names(media_info)){
                media_info$message <- "No Message"
              }


              ins <- GET(
                "https://graph.facebook.com/",
                path = paste0(mids[k], "/", "insights/"),
                query = list(
                  "metric" = paste(collapse = ", ", params),
                  "access_token" = input_FB_ids$access_token[j],
                  period = "lifetime",
                  since = intervals_FB$unix_min[i],
                  until = intervals_FB$unix_max[i]
                )
              ) %>%
                content() %>%
                toJSON() %>%
                fromJSON() %>%
                bind_rows()

              ins <- ins[-c(nrow(ins)),]%>%
                mutate(values = unlist(values), title = as.character(title)) %>%
                select(c(title, values)) %>% t() %>% data.frame()
              names(ins) <- ins[1, ]
              ins <- ins[-1, ]

              out_val <- bind_cols(media_info, ins) %>%
                mutate(across(!(1:4), as.integer), created_time = as_datetime(created_time))
              # print(paste(j,k))
              return(out_val)
            }) %>% bind_rows()
          } else {

          }
        }) %>%  rbindlist(fill = T)
        # glimpse(FB_media_interval)
        ##Save Tables####
        #FB_media_sum <- FB_media_interval %>%
          #group_by(Month = lubridate::month(FB_media_interval$created_time, label = T)) %>%
          #summarize(across(.cols = where(is.numeric), .fns = sum)) %>%
          #mutate(`Engagement Rate` = (`Lifetime Engaged Users` / `Lifetime Post Total Reach`)) %>%
          #t() %>%
          #data.frame()

        FB_media_sum <- FB_media_interval %>%
          group_by(Month = lubridate::month(FB_media_interval$created_time, label = T)) %>%
          summarize(across(.cols = where(is.numeric), .fns = sum)) %>%
          mutate(`Engagement Rate` = (`post_engaged_users` / `post_impressions_unique`)) %>%
          t() %>%
          data.frame()

        if(ncol(FB_media_sum)>1){
          names(FB_media_sum) <- c(FB_media_sum[1,])
          FB_media_sum <- FB_media_sum[-1,] %>%
            mutate(across(where(is.character), as.numeric))
          corrected_metrics <- rownames(FB_media_sum)
          corrected_metrics <- gsub("_", " ", corrected_metrics)
          corrected_metrics <- gsub("Lifetime ", "", corrected_metrics) %>%
            stringr::str_to_title()
          FB_media_sum$Change <- FB_media_sum[,2] - FB_media_sum[,1]
          FB_media_sum[rownames(FB_media_sum) !="Engagement Rate",] <- FB_media_sum[rownames(FB_media_sum) !="Engagement Rate",] %>%
            mutate(across(everything(),scales::comma,accuracy = 1))
          FB_media_sum["Engagement Rate",] <- scales::percent(as.numeric(FB_media_sum["Engagement Rate",]), accuracy = 0.01)
          FB_media_sum <- cbind(Metric = corrected_metrics, FB_media_sum)

          # glimpse(FB_media_sum)
          # FB_media_sum$Change <- 0
          # FB_media_sum <- bind_cols(rownames(FB_media_sum),FB_media_sum)
          #
          # Change <- FB_media_sum[,3] - FB_media_sum[,2]




          clientName = input_FB_ids$name[[j]]

          #need to fix this to be arbitrary
          # reportMonth = "Jul"

          #temp test section
          mainDir =
            (getwd())
          subDir =
            paste(reportMonth,
                  "/",
                  clientName,
                  sep = "")
          #Folder Prep####
          #ensure folder exists, then
          prep_FB_media_folder <-
            function(clientName,
                     reportMonth = lubridate::month(Sys.time(), label = T),
                     mainDir,
                     subDir) {
              dir.create(file.path(mainDir, subDir, "FBOutput", "Media"),
                         recursive = TRUE)

              file_p_media <- file.path(
                mainDir,
                subDir,
                "FBOutput",
                "Media",
                paste(clientName, "FB", reportMonth, "media_info.csv", sep = "_")
              )

              file_p_media_ins <- file.path(
                mainDir,
                subDir,
                "FBOutput",
                "Media",
                paste(clientName, "FB", reportMonth, "media_summary.csv", sep = "_")
              )
              write_csv(FB_media_interval, file_p_media)
              write_csv(FB_media_sum, file_p_media_ins)
            }
          prep_FB_media_folder(
            clientName = clientName,
            reportMonth = reportMonth,
            mainDir = mainDir,
            subDir = subDir
          )

          dl_media <- lapply(1:nrow(FB_media_interval), function(p){
            # if(FB_media_interval$media_type[p]=="VIDEO"){
            #   ftype <- ".mp4"
            # } else {
            #   ftype <- ".jpg"
            # }
            ftype <- ".jpg"
            file_p <- file.path(
              mainDir,
              subDir,
              "FBOutput",
              "Media",
              paste(clientName, "FB", reportMonth,as.numeric(as.POSIXct(FB_media_interval$created_time[p])) ,date(FB_media_interval$created_time[p]) , ftype, sep = "_")
            )
            download(FB_media_interval$full_picture[p], file_p, mode="wb")

          })

        }

      })
      return(FB_media)


    }

    # fbtb <- fL[c(1),]
    fbtb <- input_FB_ids
    FB_Media <-
      get_FB_media(fbtb, get_pages_fb(starting, ending)#, reportMonth
                   )

    FB_insight_page <- lapply(1:nrow(intervals), function(i) {
      params <- c("page_post_engagements", "page_consumptions_unique", "page_impressions",
                  "page_impressions_paid",
                  # "page_impressions_organic",
                  "page_impressions_viral",
                  "page_impressions_nonviral","page_impressions_unique","page_video_views", "page_fan_adds",
                  "page_fan_removes")
      ins <- GET(
        "https://graph.facebook.com/",
        path = paste0(input_FB_ids$id[j], "/", "insights/"),
        query = list(
          "metric" = paste(collapse = ", ", params),
          "access_token" = input_FB_ids$access_token[j],
          period = "day",
          since = intervals$unix_min[i],
          until = intervals$unix_max[i]
        )
      ) %>%
        content() %>%
        toJSON() %>%
        fromJSON()

      for (n in 1:length(params)) {
        ins$data$values[[n]] <- data.frame(ins$data$values[[n]])

        names(ins$data$values[[n]])[1] <- gsub("Daily ", "",unlist(ins$data$title)[[n]])
      }

      out_val <- (ins$data$values %>%
                    reduce(inner_join, by = "end_time")) %>%
        select(end_time, everything()) %>%
        mutate(end_time=as_date(as.character(end_time))-1
        ) %>%
        mutate_if(is.list, as.numeric) %>%
        mutate(`Net Likes` = `New Likes` - Unlikes
        )
      names(out_val)[1] <- "Date"

      return(out_val)
    }) %>% bind_rows()

    #Folder Prep####
    prep_FB_folders <-
      function(clientName,
               reportMonth = lubridate::month(Sys.time(), label = T),
               mainDir,
               subDir) {
        dir.create(file.path(mainDir, subDir, "FBInput"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Daily"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Faceted"),
                   recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "FBOutput", "Media"), recursive = TRUE)

        file_p <- file.path(mainDir,
                            subDir,
                            "FBInput",
                            paste(clientName, "FB", reportMonth, ".csv", sep = "_"))
        # file.copy(in_file, file_p)
        write_csv(FB_insight_page, file_p)
        return(file_p)
      }
    clientName = input_FB_ids$name[[j]]
    # reportMonth = lubridate::month(Sys.time(), label = T) %>%
    #   droplevels()
    # reportMonth = "Jul"

    #temp test section
    mainDir =
      (getwd())
    subDir =
      paste(
        reportMonth,
        "/",
        input_FB_ids$name[[j]],
        sep = ""
      )

    prep_FB_folders(
      clientName = clientName,
      reportMonth = reportMonth,
      mainDir = mainDir,
      subDir = subDir
    )

    # FB Summary ####
    # print(paste("Month Curr:", FB_insight_page$Date[1]))
    FB_sum <- FB_insight_page %>%
      mutate(Metric =lubridate::month(Date, label = T)) %>%
      group_by(Metric) %>%
      summarise(across(where(is.numeric), sum)) %>%
      mutate(`Engagement Rate` = (`Post Engagements`/`Total Reach`)) %>%
      t() %>%
      data.frame()
    # glimpse(FB_sum)
    names(FB_sum) <- FB_sum[1, ]
    FB_sum <- FB_sum[-1, ]
    FB_sum <- FB_sum %>%
      mutate_at(1:2, as.numeric)
    Change <- (FB_sum[, 2] - FB_sum[, 1])
    FB_sum$Change <- Change
    FB_sum[rownames(FB_sum) !="Engagement Rate",] <- FB_sum[rownames(FB_sum) !="Engagement Rate",] %>%
      mutate(across(everything(),scales::comma,accuracy = 1))
    FB_sum["Engagement Rate",] <- scales::percent(as.numeric(FB_sum["Engagement Rate",]), accuracy = 0.01)
    FB_sum <- cbind(Metric = rownames(FB_sum), FB_sum)

    file_sum_p <- file.path(
      mainDir,
      subDir,
      "FBOutput",
      paste(clientName, "FB_Summary", reportMonth, ".csv", sep = "_")
    )
    write_csv(FB_sum, file_sum_p)

    #FB Charts####

    #Fix Dates and Variables####
    fixDates <- function(inp) {
      allDate <- as_date(min(inp$Date):max(inp$Date))
      missingDates <- as_date(setdiff(allDate, inp$Date))
      fillDates <- data.frame(Date = missingDates)
      x <- bind_rows(inp, fillDates) %>%
        arrange(Date)
      x[is.na(x)] <- 0
      return(x)
    }

    FBTbl <- FB_insight_page %>%
      fixDates() %>%
      mutate(
        #Week = week(FBTbl$Date),
        Month = lubridate::month(Date, label = T),
        Day = day(Date),
        Weekday = lubridate::wday(Date, label = T)
      )
    FBTime <- c("Date", "Month", "Week", "Day", "Weekday")
    FBTime <- set_names(FBTime)
    FBVars <- names(FBTbl)
    FBVars <- FBVars[!FBVars %in% FBTime]
    FBVars <- set_names(FBVars)
    FBTbl <- FBTbl %>% droplevels()
    meanOfVars <- lapply(seq(levels(FBTbl$Month)), function(i) {
      x <- FBTbl %>%
        filter(FBTbl$Month == levels(FBTbl$Month)[i]) %>%
        select(all_of(FBVars))
      monthSum <- summarise_each(x, list(mean = mean))
      monthSum$Month <- levels(FBTbl$Month)[i]
      return(monthSum)
    }) %>% bind_rows()

    #Daily Charts####
    FB_charts_daily <- lapply(seq(FBVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- FBVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(FBTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(FBTbl[current_variable]), max(FBTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "FBOutput",
          "/","Daily","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })
    #Faceted Charts####
    FB_charts_faceted <- lapply(seq(FBVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- FBVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(FBTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(FBTbl[current_variable]), max(FBTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable) +
        facet_wrap(~Weekday)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "FBOutput",
          "/","Faceted","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })

    return(FB_insight_page)
  })
  names(FB_insights) <- input_FB_ids$name

  return(FB_insights)
}


In [ ]:
intervals_FB <- get_pages_fb(start_date = starting,end_date = ending)

input_FB_ids <- fL

j = 1

FB_media_interval <- lapply(1:nrow(intervals_FB), function(i){
          # print(paste("i:", i))
          media_ids <- GET(
            "https://graph.facebook.com/",
            path = paste0(input_FB_ids$id[j], "/", "posts/"),
            query = list(
              # "metric" = paste(collapse = ", ", params),
              "fields" = "attachments,insights.metric(post_impressions_unique,post_impressions,post_engaged_users)",
              "access_token" = input_FB_ids$access_token[j],
              # period = "lifetime",
              since = intervals_FB$unix_min[i],
              until = intervals_FB$unix_max[i]
            )
          ) %>%
            content() %>%
            toJSON() %>%
            fromJSON()
          mids <- media_ids$data$id %>% unlist()
          if(!is.null(mids)){

            FB_media_page <- lapply(1:length(mids), function(k) {
              # print(paste("k:", k))
              params <-
                c("post_engaged_users","post_clicks", "post_impressions", "post_impressions_unique", "post_reactions_like_total"#, "video_views"
                )

              media_info <- GET(
                "https://graph.facebook.com/",
                path = paste0(mids[k], "/"),
                query = list(
                  "fields" = "id,full_picture,message,created_time",
                  "metric" = paste(collapse = ", ", params),
                  "access_token" = input_FB_ids$access_token[j],
                  period = "lifetime",
                  since = intervals_FB$unix_min[i],
                  until = intervals_FB$unix_max[i]
                )
              ) %>%
                content() %>%
                toJSON() %>%
                fromJSON() %>%
                bind_cols()

              if(!("message") %in% names(media_info)){
                media_info$message <- "No Message"
              }


              ins <- GET(
                "https://graph.facebook.com/",
                path = paste0(mids[k], "/", "insights/"),
                query = list(
                  "metric" = paste(collapse = ", ", params),
                  "access_token" = input_FB_ids$access_token[j],
                  period = "lifetime",
                  since = intervals_FB$unix_min[i],
                  until = intervals_FB$unix_max[i]
                )
              ) %>%
                content() %>%
                toJSON() %>%
                fromJSON() %>%
                bind_rows()

              ins <- ins[-c(nrow(ins)),]%>%
                mutate(values = unlist(values), title = as.character(title)) %>%
                select(c(title, values)) %>% t() %>% data.frame()
              names(ins) <- ins[1, ]
              ins <- ins[-1, ]

              out_val <- bind_cols(media_info, ins) %>%
                mutate(across(!(1:4), as.integer), created_time = as_datetime(created_time))
              # print(paste(j,k))
              return(out_val)
            }) %>% bind_rows()
          } else {

          }
        })

In [ ]:
str(FB_media_interval)

## IG Media Functions

In [ ]:
get_IG_media <- function(input_IG_ids, intervals_IG,
                         reportMonth = "O") {
  IG_media <- lapply(1:nrow(input_IG_ids), function(j) {
    # print(paste("j:", j))
    #Media Ids for current account j
    IG_media_interval <- lapply(1:nrow(intervals_IG), function(i){
      # print(paste("i:", i))
      media_ids <- GET(
        "https://graph.facebook.com/",
        path = paste0(input_IG_ids$ig_bs_id[j], "/", "media/"),
        query = list(
          # "metric" = paste(collapse = ", ", params),
          "access_token" = main_token,
          # period = "lifetime",
          since = intervals_IG$unix_min[i],
          until = intervals_IG$unix_max[i]
        )
      ) %>%
        content() %>%
        toJSON() %>%
        fromJSON()
      mids <- media_ids$data$id %>% unlist()
      # mids <- mids[1]
      if(!is.null(mids)){
        # k=k+1
        IG_media_page <- lapply(1:length(mids), function(k) {
          # print(paste("k:", k))
          params <-
            c("impressions", "engagement", "reach", "saved"#, "video_views"
            )

          media_info <- GET(
            "https://graph.facebook.com/",
            path = paste0(mids[k], "/"),
            query = list(
              "fields" = "ig_id,id,media_url,media_type,caption,timestamp,like_count,comments_count",
              "metric" = paste(collapse = ", ", params),
              "access_token" = main_token,
              period = "lifetime",
              since = intervals_IG$unix_min[i],
              until = intervals_IG$unix_max[i]
            )
          ) %>%
            content() %>%
            toJSON() %>%
            fromJSON() %>%
            bind_cols()


          ins <- GET(
            "https://graph.facebook.com/",
            path = paste0(mids[k], "/", "insights/"),
            query = list(
              "metric" = paste(collapse = ", ", params),
              "access_token" = main_token,
              period = "lifetime",
              since = intervals_IG$unix_min[i],
              until = intervals_IG$unix_max[i]
            )
          ) %>%
            content()
          if(!(("error") %in% names(ins))){
          ins <- ins %>%
            toJSON() %>%
            fromJSON() %>%
            bind_rows() %>%
            mutate(values = unlist(values), title = as.character(title)) %>%
            select(c(title, values)) %>% t() %>% data.frame()
          names(ins) <- ins[1, ]
          ins <- ins[-1, ]
          } else {
            # dput((ins))
              # ins <- rep(0,length(params))
              # #Need to generalize this
              # names(ins) <- params %>%
              #   str_to_title() %>% data.frame()
            ins <- structure(list(Impressions = "0", Engagement = "0", Reach = "0",
                                  Saved = "0"), row.names = "values", class = "data.frame")
            }




          out_val <- bind_cols(media_info, ins) %>%
            mutate(across(!(1:6), as.integer), timestamp = as_datetime(timestamp))
          # print(paste(j,k))
          return(out_val)
        }) %>% bind_rows()
      } else {

      }
    }) %>%  rbindlist(fill = T)
    # glimpse(IG_media_interval)
    ##Save Tables####
    IG_media_sum <- IG_media_interval %>%
      group_by(Month = lubridate::month(IG_media_interval$timestamp, label = T)) %>%
      summarize(across(.cols = where(is.numeric), .fns = sum)) %>%
      mutate(`Engagement Rate` = (Engagement / Reach)) %>%
      t() %>%
      data.frame()

    if(ncol(IG_media_sum)>1){
      names(IG_media_sum) <- c(IG_media_sum[1,])
      IG_media_sum <- IG_media_sum[-1,] %>%
        mutate(across(where(is.character), as.numeric))
      corrected_metrics <- rownames(IG_media_sum)
      corrected_metrics <- gsub("_", " ", corrected_metrics) %>%
        stringr::str_to_title()
      IG_media_sum$Change <- IG_media_sum[,2] - IG_media_sum[,1]
      IG_media_sum[rownames(IG_media_sum) !="Engagement Rate",] <- IG_media_sum[rownames(IG_media_sum) !="Engagement Rate",] %>%
        mutate(across(everything(),scales::comma,accuracy = 1))
      IG_media_sum["Engagement Rate",] <- scales::percent(as.numeric(IG_media_sum["Engagement Rate",]), accuracy = 0.01)
      IG_media_sum <- cbind(Metric = corrected_metrics, IG_media_sum)

      clientName = input_IG_ids$b_name[[j]]

      #need to fix this to be arbitrary

      #temp test section
      mainDir =
        (getwd())
      subDir =
        paste(reportMonth,
              "/",
              clientName,
              sep = "")
      #Folder Prep####
      #ensure folder exists
      prep_IG_media_folder <-
        function(clientName,
                 reportMonth = lubridate::month(Sys.time(), label = T),
                 mainDir,
                 subDir) {
          dir.create(file.path(mainDir, subDir, "IGOutput", "Media"),
                     recursive = TRUE)

          file_p_media <- file.path(
            mainDir,
            subDir,
            "IGOutput",
            "Media",
            paste(clientName, "IG", reportMonth, "media_info.csv", sep = "_")
          )

          file_p_media_ins <- file.path(
            mainDir,
            subDir,
            "IGOutput",
            "Media",
            paste(clientName, "IG", reportMonth, "media_summary.csv", sep = "_")
          )
          write_csv(IG_media_interval, file_p_media)
          write_csv(IG_media_sum, file_p_media_ins)
        }
      prep_IG_media_folder(
        clientName = clientName,
        reportMonth = reportMonth,
        mainDir = mainDir,
        subDir = subDir
      )

      dl_media <- lapply(1:nrow(IG_media_interval), function(p){
        if(IG_media_interval$media_type[p]=="VIDEO"){
          ftype <- ".mp4"
        } else {
          ftype <- ".jpg"
        }
        file_p <- file.path(
          mainDir,
          subDir,
          "IGOutput",
          "Media",
          paste(
            clientName,
            "IG",
            reportMonth,
            as.numeric(as.POSIXct(IG_media_interval$timestamp[p])) ,
            ftype,
            sep = "_"
          )
        )
        download(IG_media_interval$media_url[p], file_p, mode = "wb")
      })
    }
  })
  return(IG_media)
}

## IG Insight Functions

In [ ]:
get_IG_profile_insIGhts <- function(input_IG_ids, intervals_IG,
                                    reportMonth = "JJ" ) {
  IG_insIGhts <- lapply(1:nrow(input_IG_ids), function(j) {
    IG_insIGht_page <- lapply(1:nrow(intervals_IG), function(i) {
      params <- c("impressions", "reach"#, "saved","likes", "comments",
                  #"shares", "profile_links_taps"
                  )

      ins <- GET(
        "https://graph.facebook.com/",
        path = paste0(input_IG_ids$ig_bs_id[j], "/", "insights/"),
        query = list(
          "metric" = paste(collapse = ", ", params),
          "access_token" = main_token,
          period = "day",
          since = intervals_IG$unix_min[i],
          until = intervals_IG$unix_max[i]
        )
      ) %>%
        content() %>%
        toJSON() %>%
        fromJSON()

      for (n in 1:length(params)) {
        names(ins$data$values[[n]])[1] <- params[n]
      }

      out_val <- (ins$data$values %>%
                    reduce(inner_join, by = "end_time")) %>%
        mutate(end_time=as_date(as.character(end_time))) %>%
        mutate_if(is.list, as.numeric)
      names(out_val)[2] <- "Date"
      return(out_val)
    }) %>% bind_rows()

    ##Fix Names####
    # names(IG_insIGht_page) <- input_IG_ids$b_name

    clientName = input_IG_ids$b_name[[j]]

    # reportMonth = "Jul"

    #temp test section
    mainDir =
      (getwd())
    subDir =
      paste(
        reportMonth,
        "/",
        clientName,
        sep = ""
      )
    #Folder Prep####

    prep_IG_folders <-
      function(clientName,
               reportMonth = lubridate::month(Sys.time(), label = T),
               mainDir,
               subDir) {
        dir.create(file.path(mainDir, subDir, "IGInput"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "IGOutput", "Daily"), recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "IGOutput", "Faceted"),
                   recursive = TRUE)
        dir.create(file.path(mainDir, subDir, "IGOutput", "Media"), recursive = TRUE)

        file_p <- file.path(mainDir,
                            subDir,
                            "IGInput",
                            paste(clientName, "IG", reportMonth, ".csv", sep = "_"))
        # file.copy(in_file, file_p)
        write_csv(IG_insIGht_page, file_p)
        return(file_p)
      }
    # clientName = input_IG_ids$b_name[[j]]
    # reportMonth = lubridate::month(Sys.time(), label = T) %>%
    #   droplevels()


    prep_IG_folders(
      clientName = clientName,
      reportMonth = reportMonth,
      mainDir = mainDir,
      subDir = subDir
    )

    # IG Summary ####

    IG_sum <- IG_insIGht_page %>%
      mutate(Month = lubridate::month(Date, label = T)) %>%
      group_by(Month) %>%
      summarise(across(where(is.numeric), sum)) %>%
      t() %>% data.frame()

    names(IG_sum) <- IG_sum[1, ]
    IG_sum <- IG_sum[-1, ]
    corrected_metrics <- rownames(IG_sum)
    corrected_metrics <- gsub("_", " ", corrected_metrics) %>%
      stringr::str_to_title()
    IG_sum <- IG_sum %>%
      mutate_at(1:2, as.numeric)
    Change <- (IG_sum[, 2] - IG_sum[, 1])
    IG_sum$Change <- Change
    IG_sum <- IG_sum %>%
      mutate(across(everything(),scales::comma,accuracy = 1))
    IG_sum <- cbind(Metric=corrected_metrics, IG_sum) %>%
      data.frame()
    file_sum_p <- file.path(
      mainDir,
      subDir,
      "IGOutput",
      paste(clientName, "IG_Summary", reportMonth, ".csv", sep = "_")
    )
    write_csv(IG_sum, file_sum_p)



    #IG Charts####

    #Fix Dates and Variables####
    fixDates <- function(inp) {
      allDate <- as_date(min(inp$Date):max(inp$Date))
      missingDates <- as_date(setdiff(allDate, inp$Date))
      fillDates <- data.frame(Date = missingDates)
      x <- bind_rows(inp, fillDates) %>%
        arrange(Date)
      x[is.na(x)] <- 0
      return(x)
    }

    IGTbl <- IG_insIGht_page %>%
      fixDates() %>%
      mutate(
        #Week = week(IGTbl$Date),
        Month = lubridate::month(Date, label = T),
        Day = day(Date),
        Weekday = lubridate::wday(Date, label = T)
      )
    IGTime <- c("Date", "Month", "Week", "Day", "Weekday")
    IGTime <- set_names(IGTime)
    IGVars <- names(IGTbl)
    IGVars <- IGVars[!IGVars %in% IGTime]
    IGVars <- set_names(IGVars)
    IGTbl <- IGTbl %>% droplevels()
    meanOfVars <- lapply(seq(levels(IGTbl$Month)), function(i) {
      x <- IGTbl %>%
        filter(IGTbl$Month == levels(IGTbl$Month)[i]) %>%
        select(all_of(IGVars))
      monthSum <- summarise_each(x, list(mean = mean))
      monthSum$Month <- levels(IGTbl$Month)[i]
      return(monthSum)
    }) %>% bind_rows()

    #Daily Charts####
    IG_charts_daily <- lapply(seq(IGVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- IGVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(IGTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(IGTbl[current_variable]), max(IGTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "IGOutput",
          "/","Daily","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })
    #Faceted Charts####
    IG_charts_faceted <- lapply(seq(IGVars), function(k) {
      horizontal_lines <- as.numeric()
      current_variable <- IGVars[k]
      horizontal_lines[1] <- as.numeric(meanOfVars[1, k])
      horizontal_lines[2] <- as.numeric(meanOfVars[2, k])


      ggt <- ggplot(IGTbl, aes(
        fill = Month,
        x = Day,
        y = get(current_variable)
      ))  +
        geom_bar(stat = "identity", position = "dodge") +
        scale_fill_manual(values = c("#86ccce", "#ffc33e")) +
        geom_hline(yintercept = horizontal_lines[1], color = "#86ccce") +
        geom_hline(yintercept = horizontal_lines[2], color = "#ffc33e") +
        scale_y_continuous(
          guide = guide_axis(n.dodge = 2),
          labels = scales::number_format(accuracy = 0.01),
          breaks = sort(c(
            seq(min(IGTbl[current_variable]), max(IGTbl[current_variable]), length.out = 5),
            horizontal_lines[1],
            horizontal_lines[2]
          ))
        ) +
        labs(y = current_variable) +
        facet_wrap(~Weekday)

      chartName <-
        paste(
          mainDir,
          "/",
          subDir,
          "/",
          "IGOutput",
          "/","Faceted","/",
          reportMonth,
          "_",
          current_variable,
          ".png",
          sep = ""
        )

      # ggt
      ggsave(chartName,
             plot = ggt,
             width = 9,
             height = 4)
      return(ggt)

    })


    return(IG_insIGht_page)
  })
  names(IG_insIGhts) <- input_IG_ids$b_name
  return(IG_insIGhts)
}

# Run the final Queries and Process the Data
The resultant output will be stored in the content folder in the Files explorer in the toolbar to the left.

In [ ]:
 # Start the main timer
timers <- list(all_start = proc.time(), setup_start = proc.time())

# Setup -------------------------------------------------------------------
# Load various libraries containing functions needed

# Functions for:
# Data cleaning and manipulation
library(tidyverse)
# Http requests
library(httr)
# Easily manipulating dates and times
library(lubridate)
# Data Table functions
library(data.table)
# Easily downloading items
install.packages("downloader")
library(downloader)

# Set numeric defaults to avoid scientific notation
options(scipen = 999, digits = 2)

# My FB Account ID
#main_id <- ""

# Facebook Graph API Token, set-up on developers.facebook.com
#main_token <- ""

# FB Date Pages -----------------------------------------------------------
# Since FB has a maximum time period for queries, split into pages

get_pages_fb <- function(start_date, end_date) {
  start_date <- mdy(start_date)
  end_date <- mdy(end_date)
  dv <- as_date(start_date:end_date)
  dates <- split(dv,   ceiling(seq_along(dv) / 89))
  ranges <- lapply(1:length(dates), function(i) {
    min_date = min(dates[[i]])
    max_date = max(dates[[i]])
    unix_min =  as.numeric(as.POSIXct(min_date))
    unix_max =  as.numeric(as.POSIXct(max_date)) + 86399
    min_max <-
      data.frame(
        min_date = min_date,
        max_date = max_date,
        unix_min = unix_min,
        unix_max = unix_max
      )
    return(min_max)
  }) %>% bind_rows()
  return(ranges)
}

# IG Date Pages -----------------------------------------------------------
# Since IG has a maximum time period for queries, split into pages.
# Note that this is different maximum time than FB has.
get_pages_IG <- function(start_date, end_date) {
  start_date = mdy(start_date)
  end_date = mdy(end_date)
  dv <- as_date(start_date:end_date)
  dates <- split(dv,   ceiling(seq_along(dv) / 30))
  ranges <- lapply(1:length(dates), function(i) {
    min_date = min(dates[[i]])
    max_date = max(dates[[i]])
    unix_min =  as.numeric(as.POSIXct(min_date))
    unix_max =  as.numeric(as.POSIXct(max_date)) + 86399
    min_max <-
      data.frame(
        min_date = min_date,
        max_date = max_date,
        unix_min = unix_min,
        unix_max = unix_max
      )
    return(min_max)
  }) %>% bind_rows()
  return(ranges)
}

# Load Function Scripts ---------------------------------------------------
# These Supporting script files load the custom functions
# to do the heavy lifting of downloading and processing data

# Processing page/profile insights
#source("Facebook_Page_Main_1.R")
#source("Instagram_Page_Main_1.R")

# Processing Media insights
#source("Instagram_Media_Main_1.R")
#source("Facebook_Media_Main_1.R")

# Inputs ------------------------------------------------------------------


# Start Requests ----------------------------------------------------------


# Get the IDs
#iL <- get_IG_ids(main_id, main_token)[c(1),]
#fL <- get_fb_ids(main_id, main_token)[c(1),]

timers$setup_time <- proc.time() - timers$setup_start
timers$setup_time
timers$FB_Ins_start <- proc.time()

# FB Profile --------------------------------------------------------------
# Run the FB Profile script that was already sourced
timers$FB_Ins_start <- proc.time()
FB_insight <-
  get_FB_profile_insights(fL,
                          get_pages_fb(starting, ending),
                          monthlabel)

timers$FB_Ins_time <- proc.time() - timers$FB_Ins_start
timers$FB_Ins_time

# IG Profile --------------------------------------------------------------
# Run the IG Profile script that was already sourced
timers$IG_Ins_start <- proc.time()
iL <- get_IG_ids(main_id, main_token)
IG_insight <-
  get_IG_profile_insIGhts(iL,
                          get_pages_IG(starting, ending),
                          monthlabel)

timers$IG_Ins_time <- proc.time() - timers$IG_Ins_start
timers$IG_Ins_time

# IG Media --------------------------------------------------------------
# Run the media insight script already loaded
#timers$IG_Med_start <- proc.time()
#IG_Media <-
#  get_IG_media(iL#[c(1:2,4:5,8),]
#               ,
#               get_pages_IG(starting, ending), monthlabel)


#timers$IG_Med_time <- proc.time() - timers$IG_Med_start
#timers$IG_Med_time

# Final -------------------------------------------------------------------
# End all timers
timers$all_time <- proc.time() - timers$all_start
timers$all_time

In [ ]:
iL

In [ ]:
files <- list.files(monthlabel, recursive=TRUE)
zip(paste0(monthlabel,".zip"), files=paste(monthlabel, files, sep="/"))